In [2]:
catalog.load("sharp")

[02/07/24 03:09:03] INFO     Loading data from 'sharp' (CSVDataset)...                          ]8;id=466823;file:///home/rafael/anaconda3/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=403214;file:///home/rafael/anaconda3/lib/python3.11/site-packages/kedro/io/data_catalog.py#502\502]8;;\

[02/07/24 03:09:22] WARNING  /home/rafael/anaconda3/lib/python3.11/site-packages/kedro_datasets/pan ]8;id=374051;file:///home/rafael/anaconda3/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=842210;file:///home/rafael/anaconda3/lib/python3.11/warnings.py#109\109]8;;\
                             das/csv_dataset.py:163: DtypeWarning: Columns (38,42,44,46,50,52) have                
                             mixed types. Specify dtype option on import or set low_memory=False.                  
                               return pd.read_csv(load_path, **self._load_args)                                    
                                                                                                                   

,Unnamed: 0,Timestamp,TOTUSJH,TOTBSQ,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,TOTFZ,...,BFLARE_LABEL_LOC,CFLARE_LOC,CFLARE_LABEL_LOC,MFLARE_LOC,MFLARE_LABEL_LOC,XFLARE_LOC,XFLARE_LABEL_LOC,XR_MAX,XR_QUAL,IS_TMFI
0,0,1396640160,9.198372,1.311561e+08,4.385074e+20,1.712624e+11,0.853860,2.101766e+10,2.510902e+20,-1.414599e+23,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,7.369700e-07,12,True
1,1,1396640880,23.387741,2.551019e+08,8.440929e+20,5.075056e+11,4.132148,2.805610e+11,5.437857e+20,-2.763172e+23,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,7.068000e-07,12,True
2,2,1396641600,25.812678,3.686198e+08,1.117242e+21,5.530110e+11,2.762941,1.701154e+11,7.441798e+20,-4.064209e+23,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,7.146300e-07,12,True
3,3,1396642320,39.821228,6.220775e+08,1.889669e+21,7.917290e+11,2.865347,2.051857e+11,1.190167e+21,-6.948896e+23,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,9.164400e-07,12,True
4,4,1396643040,75.015351,1.035221e+09,4.540007e+21,1.459154e+12,1.431955,1.993900e+11,1.891793e+21,-1.048875e+24,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.543400e-06,12,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888293,2888293,1320523920,10.715441,5.925972e+07,4.764918e+20,2.685866e+11,1.175094,1.240529e+11,1.427861e+20,-5.100064e+22,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.094400e-06,12,True
2888294,2888294,1320524640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.637700e-06,12,False
2888295,2888295,1320525360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.816400e-05,12,False
2888296,2888296,1320526080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.513800e-05,12,False


In [2]:
catalog.list()


[
    'sharp',
    'shuttles',
    'preprocessed_companies',
    'preprocessed_shuttles',
    'model_input_table',
    'active_modelling_pipeline.regressor',
    'candidate_modelling_pipeline.regressor',
    'parameters',
    'params:active_modelling_pipeline',
    'params:active_modelling_pipeline.model_options',
    'params:active_modelling_pipeline.model_options.test_size',
    'params:active_modelling_pipeline.model_options.random_state',
    'params:active_modelling_pipeline.model_options.features',
    'params:candidate_modelling_pipeline',
    'params:candidate_modelling_pipeline.model_options',
    'params:candidate_modelling_pipeline.model_options.test_size',
    'params:candidate_modelling_pipeline.model_options.random_state',
    'params:candidate_modelling_pipeline.model_options.features'
]